### Retrieval by FM

$$
\hat{y}(\mathbf{x}):=w_0+\sum_{i=1}^n w_i x_i+\sum_{i=1}^n \sum_{j=i+1}^n\left\langle\mathbf{v}_i, \mathbf{v}_j\right\rangle x_i x_j
$$

$$
\begin{aligned}
& \frac{1}{2} \sum_{f=1}^k\left(\left(\sum_{i=1}^n v_{i, f} x_i\right)^2-\sum_{i=1}^n v_{i, f}^2 x_i^2\right) \\
= & \frac{1}{2} \sum_{f=1}^k\left(\left(\sum_{u \in U} v_{u, f} x_u+\sum_{t \in I} v_{t, f} x_t\right)^2-\sum_{u \in U} v_{u, f}^2 x_u^2-\sum_{t \in I} v_{t, f}^2 x_t^2\right) \\
= & \frac{1}{2} \sum_{f=1}^k\left(\left(\sum_{u \in U} v_{u, f} x_u\right)^2+\left(\sum_{t \in I} v_{t, f} x_t\right)^2+2 \sum_{u \in U} v_{u, f} x_u \sum_{t \in I} v_{t, f} x_t-\sum_{u \in U} v_{u, f}^2 x_u^2-\sum_{t \in I} v_{t, f}^2 x_t^2\right)
\end{aligned}
$$

用户向量：

用户向量由两项表达式拼接得到。
* 第一项为常数 1
* 第二项是将用户相关的特征向量进行 sum pooling

$$
V_{\text {user }}=\left[1 ; \quad \sum_{u \in U} v_u x_u\right]
$$

物品向量：

用户向量由两项表达式拼接得到。
* 第一项表示物品相关特征向量的一阶、二阶特征交互。
* 第二项是将物品相关的特征向量进行 sum pooling 。

$$
V_{item}=\left[\sum_{t \in I} w_t x_t+\frac{1}{2} \sum_{f=1}^k\left(\left(\sum_{t \in I} v_{t, f} x_t\right)^2-\sum_{t \in I} v_{t, f}^2 x_t^2\right) ; \quad \sum_{t \in I} v_t x_t\right]
$$

In [1]:
import pandas as pd
import numpy as np
import warnings
import random, math, os
from tqdm import tqdm

from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
import tensorflow.keras.backend as K

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import faiss
warnings.filterwarnings('ignore')

2023-09-01 16:54:30.806691: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# 评价指标
# 推荐系统推荐正确的商品数量占用户实际点击的商品数量
def Recall(Rec_dict, Val_dict):
    '''
    Rec_dict: 推荐算法返回的推荐列表, 形式:{uid: {item1, item2,...}, uid: {item1, item2,...}, ...} 
    Val_dict: 用户实际点击的商品列表, 形式:{uid: {item1, item2,...}, uid: {item1, item2,...}, ...}
    '''
    hit_items = 0
    all_items = 0
    for uid, items in Val_dict.items():
        rel_set = items
        rec_set = Rec_dict[uid]
        for item in rec_set:
            if item in rel_set:
                hit_items += 1
        all_items += len(rel_set)

    return round(hit_items / all_items * 100, 2)

# 推荐系统推荐正确的商品数量占给用户实际推荐的商品数
def Precision(Rec_dict, Val_dict):
    '''
    Rec_dict: 推荐算法返回的推荐列表, 形式:{uid: {item1, item2,...}, uid: {item1, item2,...}, ...} 
    Val_dict: 用户实际点击的商品列表, 形式:{uid: {item1, item2,...}, uid: {item1, item2,...}, ...}
    '''
    hit_items = 0
    all_items = 0
    for uid, items in Val_dict.items():
        rel_set = items
        rec_set = Rec_dict[uid]
        for item in rec_set:
            if item in rel_set:
                hit_items += 1
        all_items += len(rec_set)

    return round(hit_items / all_items * 100, 2)

# 所有被推荐的用户中,推荐的商品数量占这些用户实际被点击的商品数量
def Coverage(Rec_dict, Trn_dict):
    '''
    Rec_dict: 推荐算法返回的推荐列表, 形式:{uid: {item1, item2,...}, uid: {item1, item2,...}, ...} 
    Trn_dict: 训练集用户实际点击的商品列表, 形式:{uid: {item1, item2,...}, uid: {item1, item2,...}, ...}
    '''
    rec_items = set()
    all_items = set()
    for uid in Rec_dict:
        for item in Trn_dict[uid]:
            all_items.add(item)
        for item in Rec_dict[uid]:
            rec_items.add(item)
    return round(len(rec_items) / len(all_items) * 100, 2)

# 使用平均流行度度量新颖度,如果平均流行度很高(即推荐的商品比较热门),说明推荐的新颖度比较低
def Popularity(Rec_dict, Trn_dict):
    '''
    Rec_dict: 推荐算法返回的推荐列表, 形式:{uid: {item1, item2,...}, uid: {item1, item2,...}, ...} 
    Trn_dict: 训练集用户实际点击的商品列表, 形式:{uid: {item1, item2,...}, uid: {item1, item2,...}, ...}
    '''
    pop_items = {}
    for uid in Trn_dict:
        for item in Trn_dict[uid]:
            if item not in pop_items:
                pop_items[item] = 0
            pop_items[item] += 1
    
    pop, num = 0, 0
    for uid in Rec_dict:
        for item in Rec_dict[uid]:
            pop += math.log(pop_items[item] + 1) # 物品流行度分布满足长尾分布,取对数可以使得平均值更稳定
            num += 1  
    return round(pop / num, 3)

# 将几个评价指标指标函数一起调用
def rec_eval(val_rec_items, val_user_items, trn_user_items):
    print('recall:',Recall(val_rec_items, val_user_items))
    print('precision',Precision(val_rec_items, val_user_items))
    print('coverage',Coverage(val_rec_items, trn_user_items))
    print('Popularity',Popularity(val_rec_items, trn_user_items)) 

In [3]:
def get_data(root_path):
    # 读取数据时，定义的列名
    rnames = ['user_id','movie_id','rating','timestamp']
    data = pd.read_csv(os.path.join(root_path, 'ratings.dat'), sep='::', engine='python', names=rnames)

    lbe = LabelEncoder()
    data['user_id'] = lbe.fit_transform(data['user_id'])
    data['movie_id'] = lbe.fit_transform(data['movie_id']) 

    # 直接这么分是不是可能会存在验证集中的用户或者商品不在训练集合中呢？那这种的操作一半是怎么进行划分
    trn_data_, val_data_, _, _ = train_test_split(data, data, test_size=0.2)

    trn_data = trn_data_.groupby('user_id')['movie_id'].apply(list).reset_index()
    val_data = val_data_.groupby('user_id')['movie_id'].apply(list).reset_index()

    trn_user_items = {}
    val_user_items = {}
    
    # 将数组构造成字典的形式{user_id: [item_id1, item_id2,...,item_idn]}
    for user, movies in zip(*(list(trn_data['user_id']), list(trn_data['movie_id']))):
        trn_user_items[user] = set(movies)

    for user, movies in zip(*(list(val_data['user_id']), list(val_data['movie_id']))):
        val_user_items[user] = set(movies)

    return trn_user_items, val_user_items, trn_data_, val_data_, data

In [10]:
# 矩阵分解模型
def MF(n_users, n_items, embedding_dim=8):
    K.clear_session()
    input_users = Input(shape=[1, ])
    users_emb = Embedding(n_users, embedding_dim)(input_users)
    
    input_movies = Input(shape=[1, ])
    movies_emb = Embedding(n_items, embedding_dim)(input_movies)
    
    users = BatchNormalization()(users_emb)
    users = Reshape((embedding_dim, ))(users)
    
    movies = BatchNormalization()(movies_emb)
    movies = Reshape((embedding_dim, ))(movies)
    
    output = Dot(1)([users, movies])
    model = Model(inputs=[input_users, input_movies], outputs=output)
    model.compile(loss='mse', optimizer='adam')
    model.summary()
    
    # 为了方便获取模型中的某些层，进行如下属性设置
    model.__setattr__('user_input', input_users)
    model.__setattr__('user_embedding', users_emb)
    model.__setattr__('movie_input', input_movies)
    model.__setattr__('movie_embedding', movies_emb)

    return model

In [11]:
# K表示最终给用户推荐的商品数量，N表示候选推荐商品为用户交互过的商品相似商品的数量
k = 80
N = 10

In [12]:
# 读取数据
root_path = '../data/ml-1m/'
trn_user_items, val_user_items, trn_data, val_data, data = get_data(root_path)

In [13]:
# 模型保存的名称
# 定义模型训练时监控的相关参数
model_path = 'mf.h5'
checkpoint = ModelCheckpoint(model_path, monitor='val_loss', verbose=1, save_best_only=True, 
                             mode='min', save_weights_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.0001, verbose=1)
earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, verbose=1, mode='min')
callbacks = [checkpoint, reduce_lr, earlystopping]

In [14]:
# 计算user和item的数量
n_users = trn_data['user_id'].max() + 1
n_items = trn_data['movie_id'].max() + 1
embedding_dim = 64 # 用户及商品的向量维度
model = MF(n_users, n_items, embedding_dim) # 训练模型

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 embedding (Embedding)       (None, 1, 64)                386560    ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, 1, 64)                237184    ['input_2[0][0]']             
                                                                                              

In [15]:
# 模型的输入是user_id和movie_id
hist = model.fit([trn_data['user_id'].values, trn_data['movie_id'].values], 
                trn_data['rating'].values, batch_size=256, epochs=1, validation_split=0.1,
                callbacks=callbacks, verbose=1, shuffle=True)

2809/2814 [============================>.] - ETA: 0s - loss: 3.6828
Epoch 1: val_loss improved from inf to 0.91740, saving model to mf.h5
2814/2814 [==============================] - 13s 4ms/step - loss: 3.6787 - val_loss: 0.9174 - lr: 0.0010


In [16]:
# 获取模型的Embedding层
user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
item_embedding_model = Model(inputs=model.movie_input, outputs=model.movie_embedding)

In [21]:
# 将验证集中的user_id进行排序,方便与faiss搜索的结果进行对应
val_uids = np.array(sorted(val_data['user_id'].unique()))
trn_items = np.array(sorted(trn_data['movie_id'].unique()))

In [22]:
# 获取训练数据的实际索引与相对索引，
# 实际索引指的是数据中原始的user_id
# 相对索引指的是，排序后的位置索引，这个对应的是faiss库搜索得到的结果索引
trn_items_dict = {}
for i, item in enumerate(trn_items):
    trn_items_dict[i] = item

In [23]:
items_dict = set(trn_data['movie_id'].unique())

In [24]:
user_embs = user_embedding_model.predict([val_uids], batch_size=256).squeeze(axis=1)
item_embs = item_embedding_model.predict([trn_items], batch_size=256).squeeze(axis=1)

15/15 [==============================] - 0s 1ms/step


In [25]:
# 使用向量搜索库进行最近邻搜索
index = faiss.IndexFlatIP(embedding_dim)
index.add(item_embs)
# ascontiguousarray函数将一个内存不连续存储的数组转换为内存连续存储的数组，使得运行速度更快。
D, I = index.search(np.ascontiguousarray(user_embs), k)

In [26]:
# 将推荐结果转换成可以计算评价指标的格式
# 选择最相似的TopN个item
val_rec = {}
for i, u in enumerate(val_uids):
    items = list(map(lambda x: trn_items_dict[x], list(I[i]))) # 先将相对索引转换成原数据中的user_id
    items = list(filter(lambda x: x not in trn_user_items[u], items))[:N] # 过滤掉用户在训练集中交互过的商品id，并选择相似度最高的前N个
    val_rec[u] = set(items) # 将结果转换成统一的形式，便于计算模型的性能指标

In [27]:
# 计算评价指标
rec_eval(val_rec, val_user_items, trn_user_items)

recall: 0.04
precision 0.13
coverage 70.65
Popularity 2.639


### Word2vec

In [1]:
from nltk.corpus import wordnet as wn
poses = { 'n':'noun', 'v':'verb', 's':'adj (s)', 'a':'adj', 'r':'adv'}
for synset in wn.synsets("good"):
  print("{}: {}".format(poses[synset.pos()],", ".join([l.name() for l in synset.lemmas()])))

noun: good
noun: good, goodness
noun: good, goodness
noun: commodity, trade_good, good
adj: good
adj (s): full, good
adj: good
adj (s): estimable, good, honorable, respectable
adj (s): beneficial, good
adj (s): good
adj (s): good, just, upright
adj (s): adept, expert, good, practiced, proficient, skillful, skilful
adj (s): good
adj (s): dear, good, near
adj (s): dependable, good, safe, secure
adj (s): good, right, ripe
adj (s): good, well
adj (s): effective, good, in_effect, in_force
adj (s): good
adj (s): good, serious
adj (s): good, sound
adj (s): good, salutary
adj (s): good, honest
adj (s): good, undecomposed, unspoiled, unspoilt
adj (s): good
adv: well, good
adv: thoroughly, soundly, good


In [2]:
panda = wn.synset("panda.n.01")
hyper = lambda s: s.hypernyms()
list(panda.closure(hyper))

[Synset('procyonid.n.01'),
 Synset('carnivore.n.01'),
 Synset('placental.n.01'),
 Synset('mammal.n.01'),
 Synset('vertebrate.n.01'),
 Synset('chordate.n.01'),
 Synset('animal.n.01'),
 Synset('organism.n.01'),
 Synset('living_thing.n.01'),
 Synset('whole.n.02'),
 Synset('object.n.01'),
 Synset('physical_entity.n.01'),
 Synset('entity.n.01')]

$$
-\frac{1}{T} \sum_{t=1}^T \sum_{\substack{-m \leq j \leq m \\ j \neq 0}} \log P\left(w_{t+j} \mid w_t ; \theta\right)
$$

In [4]:
def naiveSoftmaxLossAndGradient(
    centerWordVec,
    outsideWordIdx,
    outsideVectors,
    dataset
):
    """ Naive Softmax loss & gradient function for word2vec models

    Arguments:
    centerWordVec -- numpy ndarray, center word's embedding
                    in shape (word vector length, )
                    (v_c in the pdf handout)
    outsideWordIdx -- integer, the index of the outside word
                    (o of u_o in the pdf handout)
    outsideVectors -- outside vectors is
                    in shape (num words in vocab, word vector length) 
                    for all words in vocab (tranpose of U in the pdf handout)
    dataset -- needed for negative sampling, unused here.

    Return:
    loss -- naive softmax loss
    gradCenterVec -- the gradient with respect to the center word vector
                     in shape (word vector length, )
                     (dJ / dv_c in the pdf handout)
    gradOutsideVecs -- the gradient with respect to all the outside word vectors
                    in shape (num words in vocab, word vector length) 
                    (dJ / dU)
    """

    # centerWordVec:  (embedding_dim,1)
    # outsideVectors: (vocab_size,embedding_dim)

    scores = np.matmul(outsideVectors, centerWordVec)  # size=(vocab_size, 1)
    probs = softmax(scores)  # size=(vocab, 1)

    loss = -np.log(probs[outsideWordIdx])  # scalar

    dscores = probs.copy()  # size=(vocab, 1)
    dscores[outsideWordIdx] = dscores[outsideWordIdx] - 1  # dscores=y_hat - y
    gradCenterVec = np.matmul(outsideVectors, dscores)  # J关于vc的偏导数公式  size=(vocab_size, 1)
    gradOutsideVecs = np.outer(dscores, centerWordVec)  # J关于u的偏导数公式  size=(vocab_size, embedding_dim)

    return loss, gradCenterVec, gradOutsideVecs

In [5]:
def negSamplingLossAndGradient(
    centerWordVec,
    outsideWordIdx,
    outsideVectors,
    dataset,
    K=10
):
  
    negSampleWordIndices = getNegativeSamples(outsideWordIdx, dataset, K)
    indices = [outsideWordIdx] + negSampleWordIndices

    gradCenterVec =np.zeros(centerWordVec.shape)  # (embedding_size,1)
    gradOutsideVecs = np.zeros(outsideVectors.shape)  # (vocab_size, embedding_size)
    loss = 0.0

    u_o = outsideVectors[outsideWordIdx]  # size=(embedding_size,1)
    z = sigmoid(np.dot(u_o, centerWordVec))  # size=(1, )
    loss -= np.log(z) # 损失函数的第一部分
    gradCenterVec += u_o * (z - 1)   # J关于vc的偏导数的第一部分
    gradOutsideVecs[outsideWordIdx] = centerWordVec * (z - 1)  # J关于u_o的偏导数计算

    for i in range(K):
        neg_id = indices[1 + i]
        u_k = outsideVectors[neg_id]
        z = sigmoid(-np.dot(u_k, centerWordVec))
        loss -= np.log(z)
        gradCenterVec += u_k * (1-z)
        gradOutsideVecs[neg_id] += centerWordVec * (1 - z)


    return loss, gradCenterVec, gradOutsideVecs

### AirBnB Listing Embedding

Real-time Personalization using Embeddings for Search Ranking at Airbnb

https://zhuanlan.zhihu.com/p/133566801

https://zhuanlan.zhihu.com/p/43295545